In [1]:
"""
1.定义一个包含可训练参数的神经网络

2.迭代整个输入

3.通过神经网络处理输入

4.计算损失(loss)

5.反向传播梯度到神经网络的参数

6.更新网络的参数，典型的用一个简单的更新方法：weight = weight - learning_rate *gradient
"""

'\n1.定义一个包含可训练参数的神经网络\n\n2.迭代整个输入\n\n3.通过神经网络处理输入\n\n4.计算损失(loss)\n\n5.反向传播梯度到神经网络的参数\n\n6.更新网络的参数，典型的用一个简单的更新方法：weight = weight - learning_rate *gradient\n'

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [3]:
class Net (nn.Module):
    def __init__(self):
        super(Net, self).__init__ ()
        self.conv1 = nn.Conv2d (1, 6, 5)
        self.conv2 = nn.Conv2d (6, 16, 5)
        self.fc1 = nn.Linear (16*5*5, 120)
        self.fc2 = nn.Linear (120, 84)
        self.fc3 = nn.Linear (84, 10)
    def forward (self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d (F.relu(self.conv2(x)), 2)
        # view 类似与tf的reshape
        x = x.view (-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    def num_flat_features(self, x):
        # 去除维度，1可能是数据量
        size = x.size ()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()

In [4]:
print (net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
params = list (net.parameters())

In [6]:
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [7]:
# 使用一个例子
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0198, -0.0197,  0.0709, -0.0163,  0.0684, -0.0525, -0.0864,  0.0005,
          0.0098,  0.1199]], grad_fn=<AddmmBackward>)


In [8]:
# 调用反向传播
net.zero_grad()
out.backward(torch.randn(1, 10))

In [9]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
print (target)
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor([ 0.1844,  0.3798,  0.3540, -0.0218,  1.3360,  1.0148,  1.7709,  0.7548,
         1.9860,  0.6084])
tensor(1.1175, grad_fn=<MseLossBackward>)


In [10]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [11]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0048, -0.0122, -0.0265, -0.0046, -0.0316,  0.0126])


In [ ]:

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update